# INF582 AXA Challenge

## Initialisation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Load main librairies
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sb
import datetime as dt
import itertools
import random

from sklearn.ensemble import GradientBoostingRegressor
from sklearn import __version__
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import train_test_split

pd.set_option('display.max_columns', 500)

## Load data

In [ ]:
# Load data
submission = pd.read_csv("data/submission.txt", sep='\t')


iter_csv = pd.read_csv("data/train_2011_2012.csv", sep=';',
                            na_values=['A Définir', 'A DEFINIR', '9999-12-31 00:00:00.000'],
                            usecols=['DATE','WEEK_END','DAY_WE_DS','TPER_TEAM','ASS_ASSIGNMENT','CSPL_RECEIVED_CALLS'],
                           iterator=True, 
                           chunksize=1000)
training_data = pd.concat([chunk[chunk['ASS_ASSIGNMENT'].isin(submission.ASS_ASSIGNMENT.unique())] for chunk in iter_csv])


training_data.head()

## Clean data

In [ ]:
# Aggregate the calls
training_data = training_data.groupby([col for col in training_data.columns if not col == 'CSPL_RECEIVED_CALLS']).sum().reset_index()

In [ ]:
#Extract the time slot from date
def get_time(date):
    time = dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S.000').time()
    return time.hour * 3600 + time.minute * 60 + time.second

def get_month(date):
    month = dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S.000').month
    return month

epoch = dt.datetime.utcfromtimestamp(0)

def unix_time_s(date_):
    return (date_ - epoch).total_seconds()

def get_tse(date):  #time since epoch
    month = unix_time_s(dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S.000'))
    return month

#Join weather
#training_data = pd.merge(training_data, meteo, how='left', on=['DATE'])

#Deal with NAs. Would be better to replace with average
#training_data.fillna(0,inplace = True)

#Ca a l'air de fonctionner mais mieux vaut ne pas regarder trop près ce qui se passe...
#training_data.set_index('DATE')
#training_data.interpolate(method='index',inplace=True)

#Assign a number to the day of the week
day_to_num_dict = {j:i for i,j in enumerate(['Lundi','Mardi','Mercredi','Jeudi','Vendredi','Samedi','Dimanche'])}

training_data['TIME'] = training_data.DATE.map(get_time)
training_data['MONTH'] = training_data.DATE.map(get_month)
training_data['TSE'] = training_data.DATE.map(get_tse)
training_data['WEEK_DAY'] = training_data.DAY_WE_DS.map(day_to_num_dict)
training_data['NIGHT'] = (training_data.TPER_TEAM == "Nuit") * 1

# Remove obsolete columns
training_data = training_data[[col for col in training_data.columns if not col in ['DATE','DAY_WE_DS','TPER_TEAM']]]

In [ ]:
#Convert the different ASS_ASSIGNMENTs to booleans
for value in submission.ASS_ASSIGNMENT.unique():
    training_data["ASS_ASSIGNMENT_"+value] = (training_data.ASS_ASSIGNMENT == value) * 1
    
# Remove obsolete column
training_data = training_data[[col for col in training_data.columns if not col == 'ASS_ASSIGNMENT']]

In [ ]:
training_data.head()

## View main statistics

In [ ]:
training_data.describe()

##  A simple predictor

Lets try building a tree-based boosting predictor with very few attributes just to see how it goes.
This predictor will only predict the number of calls received during a given time stamp (e.g. in a 30 minutes slot) and day of week.

Note that the date is not relevant for regression, but we can extract some relevant information from it: day of the week, time slot, and if it is a week-end or not.

Also, for some reason the data for a given ASS_ASSIGNMENT and DATE is sometimes split, so we have to aggregate it.

In [ ]:
output_cols = ['CSPL_RECEIVED_CALLS']
input_cols = [col for col in training_data.columns if not col in output_cols]

In [ ]:
#Now create the gradient boosting regressor

est = GradientBoostingRegressor(n_estimators=60,learning_rate=0.7,
                               max_depth=10
                               )


est.fit(training_data[input_cols],training_data[output_cols].values.ravel())
best_est=est



# Prediction and submission

In [ ]:
test_data = submission.copy()

#Deal with NAs
test_data.set_index('DATE')
test_data.interpolate(method='index',inplace=True)


def get_weekday(date):
    return dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S.000').weekday()

test_data['TIME'] = test_data.DATE.map(get_time)
test_data['MONTH'] = test_data.DATE.map(get_month)
test_data['TSE'] = test_data.DATE.map(get_tse)
test_data['WEEK_DAY'] = test_data.DATE.map(get_weekday)
test_data['NIGHT'] = (np.logical_or(test_data.TIME >= (23*3600 + 30*60),
                                    test_data.TIME <  (7*3600  + 30*60))) * 1
test_data['WEEK_END'] = test_data.WEEK_DAY.isin([5, 6]) * 1

# Convert the different ASS_ASSIGNMENTs to booleans
for value in submission.ASS_ASSIGNMENT.unique():
    test_data["ASS_ASSIGNMENT_"+value] = (test_data.ASS_ASSIGNMENT == value) * 1

test_data = test_data[input_cols]

In [ ]:
submission.prediction = best_est.predict(test_data)
submission.head()

In [ ]:
#no negative number of call
submission['prediction'] = submission['prediction'].clip(0,None)


In [ ]:
submission.head()

In [ ]:
submission.to_csv("data/output.txt", sep='\t', index=False)